<a href="https://colab.research.google.com/github/upep/weather/blob/main/Wheather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

! pip install -q kaggle  #install the kaggle library
! mkdir ~/.kaggle          #create a directory where the API Key is going to live
! cp kaggle.json ~/.kaggle/. #copy the API key to the directory
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download muthuj7/weather-dataset
!unzip  weather-dataset.zip


df = pd.read_csv("weatherHistory.csv")
df=df.drop(columns=['Loud Cover'], errors='ignore') # Added errors='ignore' to handle cases where the column might not exist

df.head()
df.info()
df.describe()



mkdir: cannot create directory ‘/root/.kaggle’: File exists
/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: ` cp kaggle(1).json ~/.kaggle/. #copy the API key to the directory'
/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: ` chmod 600 ~/.kaggle/kaggle(1).json'
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 1734, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~

FileNotFoundError: [Errno 2] No such file or directory: 'weatherHistory.csv'

In [ ]:
#checking for missing values
null_counts = df.isnull().sum()
print(null_counts)

non_numeric_cols=df.select_dtypes(exclude=['number']).columns

new_df=df.copy() #the new dataset

#filling misssing values
for col in new_df.columns:
  if col not in non_numeric_cols:
    new_df[col]=new_df[col].fillna(new_df[col].mean())


#cleaning outliers
cols_to_clean = ['Temperature (C)', 'Apparent Temperature (C)', 'Humidity', 'Wind Speed (km/h)', 'Wind Bearing (degrees)', 'Visibility (km)', 'Pressure (millibars)']
Q1 = new_df[cols_to_clean].quantile(0.25)
Q3 = new_df[cols_to_clean].quantile(0.75)
IQR=Q3-Q1

lower_bound=Q1-1.5*IQR
upper_bound=Q3+1.5*IQR

#outlier_df is the new dataset name
outlier_df=new_df[(new_df[cols_to_clean]>=lower_bound)
&(new_df[cols_to_clean]<=upper_bound)].all(axis=1)

#Remove duplicates
#cleaned_df is the new data set
cleaned_df=new_df[outlier_df].drop_duplicates()

print(cleaned_df.columns)



Formatted Date                0
Summary                       0
Precip Type                 517
Temperature (C)               0
Apparent Temperature (C)      0
Humidity                      0
Wind Speed (km/h)             0
Wind Bearing (degrees)        0
Visibility (km)               0
Pressure (millibars)          0
Daily Summary                 0
dtype: int64
Index(['Formatted Date', 'Summary', 'Precip Type', 'Temperature (C)',
       'Apparent Temperature (C)', 'Humidity', 'Wind Speed (km/h)',
       'Wind Bearing (degrees)', 'Visibility (km)', 'Pressure (millibars)',
       'Daily Summary'],
      dtype='object')


In [ ]:
from sklearn.preprocessing import OneHotEncoder
#encode nominal data (precip type)
ohe=OneHotEncoder(handle_unknown='ignore',sparse_output=False).set_output(transform='pandas')
ohetransformer=ohe.fit_transform(cleaned_df[['Precip Type']])
ohetransformer

,Precip Type_rain,Precip Type_snow,Precip Type_nan
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0
...,...,...,...
96448,1.0,0.0,0.0
96449,1.0,0.0,0.0
96450,1.0,0.0,0.0
96451,1.0,0.0,0.0


In [ ]:
#add encoded columns back to dataset
df_encoded=pd.concat([cleaned_df,ohetransformer],axis=1).drop(columns=['Precip Type'])
df_encoded.head()

,Formatted Date,Summary,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Pressure (millibars),Daily Summary,Precip Type_rain,Precip Type_snow,Precip Type_nan
0,2006-04-01 00:00:00.000 +0200,Partly Cloudy,9.472222,7.388889,0.89,14.1197,251.0,15.8263,1015.13,Partly cloudy throughout the day.,1.0,0.0,0.0
1,2006-04-01 01:00:00.000 +0200,Partly Cloudy,9.355556,7.227778,0.86,14.2646,259.0,15.8263,1015.63,Partly cloudy throughout the day.,1.0,0.0,0.0
2,2006-04-01 02:00:00.000 +0200,Mostly Cloudy,9.377778,9.377778,0.89,3.9284,204.0,14.9569,1015.94,Partly cloudy throughout the day.,1.0,0.0,0.0
3,2006-04-01 03:00:00.000 +0200,Partly Cloudy,8.288889,5.944444,0.83,14.1036,269.0,15.8263,1016.41,Partly cloudy throughout the day.,1.0,0.0,0.0
4,2006-04-01 04:00:00.000 +0200,Mostly Cloudy,8.755556,6.977778,0.83,11.0446,259.0,15.8263,1016.51,Partly cloudy throughout the day.,1.0,0.0,0.0


In [ ]:
ohetransformer_summary=ohe.fit_transform(cleaned_df[['Summary']])
ohetransformer_summary

df_encoded_summary=pd.concat([df_encoded,ohetransformer_summary],axis=1).drop(columns=['Summary'])
df_encoded_summary.head()

,Formatted Date,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Pressure (millibars),Daily Summary,Precip Type_rain,...,Summary_Mostly Cloudy,Summary_Overcast,Summary_Partly Cloudy,Summary_Rain,Summary_Windy,Summary_Windy and Dry,Summary_Windy and Foggy,Summary_Windy and Mostly Cloudy,Summary_Windy and Overcast,Summary_Windy and Partly Cloudy
0,2006-04-01 00:00:00.000 +0200,9.472222,7.388889,0.89,14.1197,251.0,15.8263,1015.13,Partly cloudy throughout the day.,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2006-04-01 01:00:00.000 +0200,9.355556,7.227778,0.86,14.2646,259.0,15.8263,1015.63,Partly cloudy throughout the day.,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2006-04-01 02:00:00.000 +0200,9.377778,9.377778,0.89,3.9284,204.0,14.9569,1015.94,Partly cloudy throughout the day.,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2006-04-01 03:00:00.000 +0200,8.288889,5.944444,0.83,14.1036,269.0,15.8263,1016.41,Partly cloudy throughout the day.,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2006-04-01 04:00:00.000 +0200,8.755556,6.977778,0.83,11.0446,259.0,15.8263,1016.51,Partly cloudy throughout the day.,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#df_encoded_summary['Formatted Date'] = pd.to_datetime(df_encoded_summary['Formatted Date'], format="%Y-%m-%d %H:%M:%S.%f %z")

df_encoded_summary['Formatted date'] = pd.to_datetime(df_encoded_summary['Formatted Date'],utc=True, errors='raise')
print(df_encoded_summary['Formatted date'].head())
print(df_encoded_summary['Formatted date'].dtype)

df_encoded_summary['month'] = df_encoded_summary['Formatted date'].dt.month
df_encoded_summary['day'] = df_encoded_summary['Formatted date'].dt.day
df_encoded_summary['hour'] = df_encoded_summary['Formatted date'].dt.hour


0   2006-03-31 22:00:00+00:00
1   2006-03-31 23:00:00+00:00
2   2006-04-01 00:00:00+00:00
3   2006-04-01 01:00:00+00:00
4   2006-04-01 02:00:00+00:00
Name: Formatted date, dtype: datetime64[ns, UTC]
datetime64[ns, UTC]


In [ ]:
#df_encoded_summary = df_encoded_summary.drop(columns=['Formatted date'])
df_encoded_summary.head()

,Formatted Date,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Pressure (millibars),Daily Summary,Precip Type_rain,...,Summary_Rain,Summary_Windy,Summary_Windy and Dry,Summary_Windy and Foggy,Summary_Windy and Mostly Cloudy,Summary_Windy and Overcast,Summary_Windy and Partly Cloudy,month,day,hour
0,2006-04-01 00:00:00.000 +0200,9.472222,7.388889,0.89,14.1197,251.0,15.8263,1015.13,Partly cloudy throughout the day.,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,31,22
1,2006-04-01 01:00:00.000 +0200,9.355556,7.227778,0.86,14.2646,259.0,15.8263,1015.63,Partly cloudy throughout the day.,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,31,23
2,2006-04-01 02:00:00.000 +0200,9.377778,9.377778,0.89,3.9284,204.0,14.9569,1015.94,Partly cloudy throughout the day.,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,0
3,2006-04-01 03:00:00.000 +0200,8.288889,5.944444,0.83,14.1036,269.0,15.8263,1016.41,Partly cloudy throughout the day.,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,1
4,2006-04-01 04:00:00.000 +0200,8.755556,6.977778,0.83,11.0446,259.0,15.8263,1016.51,Partly cloudy throughout the day.,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,2


In [ ]:
print(df_encoded_summary.columns)

Index(['Formatted Date', 'Temperature (C)', 'Apparent Temperature (C)',
       'Humidity', 'Wind Speed (km/h)', 'Wind Bearing (degrees)',
       'Visibility (km)', 'Pressure (millibars)', 'Daily Summary',
       'Precip Type_rain', 'Precip Type_snow', 'Precip Type_nan',
       'Summary_Breezy', 'Summary_Breezy and Dry', 'Summary_Breezy and Foggy',
       'Summary_Breezy and Mostly Cloudy', 'Summary_Breezy and Overcast',
       'Summary_Breezy and Partly Cloudy', 'Summary_Clear',
       'Summary_Dangerously Windy and Partly Cloudy', 'Summary_Drizzle',
       'Summary_Dry', 'Summary_Dry and Mostly Cloudy',
       'Summary_Dry and Partly Cloudy', 'Summary_Foggy',
       'Summary_Humid and Mostly Cloudy', 'Summary_Humid and Overcast',
       'Summary_Humid and Partly Cloudy', 'Summary_Light Rain',
       'Summary_Mostly Cloudy', 'Summary_Overcast', 'Summary_Partly Cloudy',
       'Summary_Rain', 'Summary_Windy', 'Summary_Windy and Dry',
       'Summary_Windy and Foggy', 'Summary_Windy and 

In [ ]:
#normalizaton
#defines a list of numerical features that you want to normalize in the DataFrame.
numerical_features = ['Temperature (C)', 'Apparent Temperature (C)',
                     'Humidity', 'Wind Speed (km/h)',
                     'Wind Bearing (degrees)', 'Visibility (km)',
                     'Pressure (millibars)', 'Precip Type_rain', 'Precip Type_snow', 'Precip Type_nan',
                      'Summary_Breezy', 'Summary_Breezy and Dry', 'Summary_Breezy and Foggy',
                     'Summary_Breezy and Mostly Cloudy', 'Summary_Breezy and Overcast',
                      'Summary_Breezy and Partly Cloudy', 'Summary_Clear',
                     'Summary_Dangerously Windy and Partly Cloudy', 'Summary_Drizzle',
                    'Summary_Dry', 'Summary_Dry and Mostly Cloudy',
                     'Summary_Dry and Partly Cloudy', 'Summary_Foggy',
                   'Summary_Humid and Mostly Cloudy', 'Summary_Humid and Overcast',
                    'Summary_Humid and Partly Cloudy', 'Summary_Light Rain',
                   'Summary_Mostly Cloudy', 'Summary_Overcast', 'Summary_Partly Cloudy',
                   'Summary_Rain', 'Summary_Windy', 'Summary_Windy and Dry',
                  'Summary_Windy and Foggy', 'Summary_Windy and Mostly Cloudy',
                  'Summary_Windy and Overcast', 'Summary_Windy and Partly Cloudy',
                  'month', 'day', 'hour']
# creates a new DataFrame  that contains only the selected numerical features from the original
normalized_df=df_encoded_summary[numerical_features]

#Min-Max scaling rescales the features to a range between 0 and 1 based on the minimum and maximum values of each feature.
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(normalized_df)

# concatenates the original DataFrame  with the new normalized DataFrame,the result is a final DataFrame  that includes all original non-normalized features and the newly normalized numerical features.
df_normalized = pd.DataFrame(normalized_data, columns=numerical_features, index=df_encoded_summary.index)
df_final = pd.concat([df_encoded_summary.drop(columns=numerical_features + ['Formatted Date']+['Daily Summary']), df_normalized], axis=1)
  #Display the final result:
print(df_final.info())


<class 'pandas.core.frame.DataFrame'>
Index: 93421 entries, 0 to 96452
Data columns (total 40 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Temperature (C)                              93421 non-null  float64
 1   Apparent Temperature (C)                     93421 non-null  float64
 2   Humidity                                     93421 non-null  float64
 3   Wind Speed (km/h)                            93421 non-null  float64
 4   Wind Bearing (degrees)                       93421 non-null  float64
 5   Visibility (km)                              93421 non-null  float64
 6   Pressure (millibars)                         93421 non-null  float64
 7   Precip Type_rain                             93421 non-null  float64
 8   Precip Type_snow                             93421 non-null  float64
 9   Precip Type_nan                              93421 non-null  float64
 10  Sum

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

#split into training and testing
precip_encoded_columns=['Precip Type_rain','Precip Type_snow','Precip Type_nan']
x=df_final.drop(columns=['Temperature (C)','Wind Speed (km/h)']+precip_encoded_columns)
y=df_final[precip_encoded_columns+['Temperature (C)','Wind Speed (km/h)']]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)


In [ ]:

# Initialize and train the RandomForestRegressor
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)  # You can adjust n_estimators
rf_regressor.fit(x_train, y_train)

# Make predictions on the test set
y_pred = rf_regressor.predict(x_test)

# Evaluate the model (example: using R-squared)
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")



R-squared: 0.7314555612652615


In [ ]:
rf=RandomForestRegressor(n_estimators=200,random_state=0)
rf.fit(x_train,y_train)

y_pred=rf.predict(x_test)

In [ ]:
#Evaluation:
#make prediction
y_pred=rf.predict(x_test)

#Calcute
mae=mean_absolute_error(y_test,y_pred)
mse=mean_squared_error(y_test,y_pred)
r2=r2_score(y_test,y_pred)

#print results
print("Mean Absolute Error:",mae)
print("Mean Squared Error:",mse)
print("R-squared:",r2)

NameError: name 'rf' is not defined